Este archivo es para implementar algunos algoritmos de aprendizaje por refuerzo y aplicarlos utilizando Open AI gym.

Se van a probar los siguientes 2 entornos (environments), de acuerdo con los resultados presentados en el articulo de Diego:

Pendulum-v0 y Hopper-v2

Tareas (Puede cambiar):
1. Terminar el curso de David Silver. (Completo: 100%)
2. Implementar con table lookup. (En progreso: Ya medio funciona (por ahi un 80%))
3. Implementar con DQN. (Sin comenzar)
4. Implementar con Simglucose (Sin comenzar)
5. Implementar con el trabajo de Diego.

# Librerías

In [1]:
import gym
import time
import numpy as np
import progressbar

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Table Lookup

Pendulum-v0<br /> 
Utilizando el algoritmo de Sarsa.<br />
Está como chambón pero funciona.

In [ ]:
def darIndice(pS):
    return int(round(pS, 1)*10+10)

def escogerAccion(pS1, pS2, pS3):
    num = np.random.random()
    
    if(num <= epsilon):
        ind = np.random.randint(0,41)
        return ind
    else:
        ind = np.where(tablaQ[pS1, pS2, pS3, :] == np.amax(tablaQ[pS1, pS2, pS3, :]))
        ind = ind[0]
        
        if(len(ind) > 1):
            return np.random.choice(ind)
        else:
            return ind
    
    return accion

In [ ]:
envName = 'Pendulum-v0'

g = 0.9
alpha = 0.5
epsilon = 0.1 #Entre 0 y 1
maxIter = 1500000
ventana = 100
epsilonIterLimit = maxIter*0.9

tablaQ = np.ones((21,21,17,41))*-16
rewards = np.zeros(int(maxIter/10))

env = gym.make(envName)
observation = env.reset()

s1 = darIndice(observation[0])
s2 = darIndice(observation[1])
s3 = int(round(observation[2], 0) + 8)
a = escogerAccion(s1, s2, s3)

suma = 0

for i in progressbar.progressbar(range(maxIter)):
    
    if(i > maxIter-2000):
        env.render()
    
    observation, reward, done, info = env.step([(a-20)/10.0])
    observation = np.reshape(observation, (3,))
    
    s1p = darIndice(observation[0])
    s2p = darIndice(observation[1])
    s3p = int(round(observation[2], 0) + 8)
    
    ap = escogerAccion(s1p, s2p, s3p)
    
    r = reward #Por ahora
    suma+=r
    
    if(i % ventana == 0):
        rewards[int(i/ventana)] = suma/ventana
        suma = 0
    
    tablaQ[s1, s2, s3, a] = tablaQ[s1, s2, s3, a] + alpha*(r + g*tablaQ[s1p, s2p, s3p, ap] - tablaQ[s1,s2,s3,a])
    
    s1 = s1p
    s2 = s2p
    s3 = s3p
    a = ap
    
    if(i > epsilonIterLimit):
        epsilon = 0
    
env.close()

In [ ]:
for i in progressbar.progressbar(range(2000)):
    env.render()
    
    observation, reward, done, info = env.step([(a-20)/10.0])
    observation = np.reshape(observation, (3,))
    
    s1p = darIndice(observation[0])
    s2p = darIndice(observation[1])
    s3p = int(round(observation[2], 0) + 8)
    
    ap = escogerAccion(s1p, s2p, s3p)
    
    r = reward #Por ahora
    suma+=r
    
    if(i % ventana == 0):
        rewards[int(i/ventana)] = suma/ventana
        suma = 0
    
    tablaQ[s1, s2, s3, a] = tablaQ[s1, s2, s3, a] + alpha*(r + g*tablaQ[s1p, s2p, s3p, ap] - tablaQ[s1,s2,s3,a])
    
    s1 = s1p
    s2 = s2p
    s3 = s3p
    a = ap
    
env.close()

# DQN

In [2]:
#Creando red neuronal
def crearModelo(n1 = 40, n2 = 41, learning_rate = 0.01):
    modelo = Sequential()
    
    modelo.add(Dense(n1, activation = 'relu', input_dim = 3))
    modelo.add(Dense(n2, activation = 'linear'))
    
    sgd = optimizers.sgd(lr = learning_rate)
    
    modelo.compile(loss = 'mean_squared_error', optimizer = sgd, metrics = ['accuracy'])
    
    return modelo

def escogerAccion(pModelo, pX):
    num = np.random.random()
    
    if(num <= epsilon):
        ind = np.random.randint(0,41)
        return ind
    else:
        ind = np.argmax(pModelo.predict(pX.T))
        return ind

In [37]:
envName = 'Pendulum-v0'

modelo = crearModelo(learning_rate = 0.001)

gamma = 0.99
alpha = 0.35
epsilon = 1#Entre 0 y 1
epsilonDecay = 0.999

episodes = 30000
ventana = 100
contadores = np.zeros((1,41))

rewards = np.zeros(int(episodes/ventana))

env = gym.make(envName)
observation = env.reset()

x = observation
x = np.reshape(x, (3,1))

a = escogerAccion(modelo, x)

suma = 0

for i in progressbar.progressbar(range(episodes)):
    
    if(i > episodes-1000):
        env.render()
    
    contadores[0,a] += 1
    observation, reward, done, info = env.step([(a-20.0)/10.0])
    
    xp = observation
    xp = np.reshape(xp, (3,1))
    
    r = reward #Por ahora
    suma+=r
    
    if(i % ventana == 0):
        rewards[int(i/ventana)] = suma/ventana
        suma = 0
    
    qValores = modelo.predict(xp.T)
    maxQvalor = np.max(qValores)
    
    #Aqui hay que calcular el maximo
    target = r+gamma*maxQvalor
    
    y = modelo.predict(x.T)
    
    y[0,a] = target
    
    modelo.fit(x.T, y, verbose = 0)
    
    x = xp
    a = escogerAccion(modelo, x)
    
    epsilon = epsilon*epsilonDecay
    
env.close()

100% (30000 of 30000) |##################| Elapsed Time: 0:01:46 Time:  0:01:46


In [19]:
testx = np.array([15,5,1])
testx = np.reshape(testx, (-1, 1))

modelo.predict(np.transpose(testx))

array([[ 0.5639679 , -0.49619234, -2.2416847 , -2.596084  , -0.7829679 ,
         0.5874912 , -1.4217947 ,  2.0881338 , -4.469524  , -1.830524  ,
        -0.41096663, -2.9731283 , -0.25903472,  0.5550725 ,  0.12394243,
        -2.26297   ,  2.0600157 , -1.8249401 , -1.2904257 , -2.6254513 ,
        -1.1714789 , -1.1079853 , -0.11576195, -0.67618024, -3.3362992 ,
         0.11744235, -2.263774  ,  0.8623138 ,  2.1498265 , -0.2982781 ,
        -1.4348308 ,  0.5167521 ,  0.5907148 , -1.3623844 ,  3.1878474 ,
        -2.2512453 , -2.4177856 , -0.50901246, -2.4590547 , -1.579963  ,
        -2.7358332 ]], dtype=float32)

In [36]:
epsilon

1.4995915609979532e-22

In [34]:
testx = np.array([1,1,0])
testx = np.reshape(testx, (-1,1))
escogerAccion(modelo, testx)

14

In [23]:
contadores

array([[43077.,  3275.,  1421.,  1897.,  2223.,  1334.,  1636.,  1681.,
         1472.,  1734.,  1859.,  1247.,  1852.,  1808.,  1403.,  1201.,
         1504.,  1151.,  1100.,  1153.,  1195.,  1326.,  1099.,  1011.,
         1867.,  1264.,  1137.,  1333.,  1205.,  1591.,  1261.,  1278.,
         1084.,  1023.,   942.,  1228.,  1645.,  1029.,   987.,  1125.,
         1342.]])

In [21]:
0.9999**50000

0.006736262610603238

In [ ]:
import gym
env = gym.make('Pendulum-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(10):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

In [ ]:
print(env.observation_space)
print(env.action_space)

In [ ]:
print(env.observation_space.high)
print(env.observation_space.low)

In [ ]:
np.random.random()

In [ ]:
np.random.randint(0,6)

In [ ]:
test = np.array([10,22,50,-1,50])

In [ ]:
test3 = np.where(test == np.amax(test))

In [ ]:
len(test3[0])

In [ ]:
rewards[10/10]

In [43]:
rewards[300]

0.0

-7.504401749226135